<a href="https://colab.research.google.com/github/teehanming/jesse/blob/master/jesse_x_Colab_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick Tutorial

This colab is to sync between your Colab & Goole Drive and train using Google Colab CPU

> **Standard Account**: You will usually get 2 Cores


> **Pro Account**: I'm able to get 40 cores !! Just $10/month.  Pick **TPU** when choosing Hardware Accelerator


###  How to Start
1. Upload your jesseData to drive
2. Default folder Directory: jesse.trade/jesseData
3. Run those cells!

###Caution
* Colab Pro can stay connected for up to 24 hours, and idle timeouts are relatively lenient. Durations are not guaranteed, though, and idle timeouts may sometimes vary. [Colab Pro](https://colab.research.google.com/signup?)

* In the free version of Colab, notebooks can run for at most 12 hours, and idle timeouts are much stricter than in Colab Pro. [More info](https://research.google.com/colaboratory/faq.html#:~:text=How%20long%20can%20notebooks%20run,or%20based%20on%20your%20usage.) 


###Extra Tips

1.   To sync your local pc + drive + colab: [download drive for desktop](https://www.google.com/drive/download/)




###Credit:
*   Written by Didi27
*   Advised by Piddy



# Google Drive Mount

In [ ]:
#@title Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


# Setup

In [ ]:
#@title 1. Install jesse.trade requirement
!source <(curl -fsSL https://raw.githubusercontent.com/teehanming/stack-installer/master/ubuntu-18.04.sh) -y

In [ ]:
#@title 2. Setup details
from os.path import join  

#@markdown Path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'My Drive/jesse.trade/' #@param {type:"string"}


#@markdown Your jesse bot name
bot_name = "YOUR_BOT_NAME" #@param {type:"string"}

#-----------------------
# No need change below
#------------------------
ROOT = '/content/drive'
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path 
!mkdir "{PROJECT_PATH}"    



In [ ]:
#@title 3. Sync Drive to Colab
#@markdown - Upload your jesseData to your Google Drive with same directory as "MY_GOOGLE_DRIVE_PATH" if you already have one.

#@markdown - Skip if you want to start new

!rsync -aP/ "{PROJECT_PATH}"/*  ./   # use remote sync to copy from google drive to local runtime google colab


# Import Data


In [ ]:
#@title Import Candles
%cd "/content/jesseData/{bot_name}"
Exchange = 'Binance' #@param ["Binance", "Coinbase Pro", "Bitfinex"]
Coin_pair = 'BTC-USDT' #@param {type:"string"}
start_date = '2018-09-07' #@param {type:"date"}
!jesse import-candles '{Exchange}' '{Coin_pair}' '{start_date}' --skip-confirmation

# Production

In [ ]:
#@title Check Colab CPU

import psutil, os, time


!lscpu |grep 'Model name'
!lscpu | grep 'Socket(s):'
!lscpu | grep 'Core(s) per socket:'
!lscpu | grep 'Thread(s) per core'
!lscpu | grep "MHz"
print('Total CPU Core:', psutil.cpu_count()) 


Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Socket(s):           1
Core(s) per socket:  20
Thread(s) per core:  2
CPU MHz:             2299.998
Total CPU Core: 40


In [ ]:
#@title Backtest
#@markdown Date: YYYY/MM/DD
%cd "/content/jesseData/{bot_name}"
b_start_date = '2021-01-01' #@param {type:"date"}
b_end_date = '2021-03-30' #@param {type:"date"}
print("Starting Backtesting...")
!jesse backtest --csv --tradingview --full-reports --json '{b_start_date}' '{b_end_date}'

/content/jesseData/didi27_bot
Starting Backtesting...
loading candles...
 CANDLES              |
----------------------+--------------------------
 period               |    88 days (2.93 months)
 starting-ending date | 2021-01-01 => 2021-03-30


 exchange   | symbol   | timeframe   | strategy     | DNA
------------+----------+-------------+--------------+-------
 Binance    | BTC-USDT | 15m         | adaptive_hma |


Executing simulation...  [####################################]  100%          
Executed backtest simulation in:  20.11 seconds
INFO:numexpr.utils:Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


 METRICS                         |
---------------------------------+---------------------------------
 Total Closed Trades             |                              95
 Total Net Profit                |            -3,050.4255 (-30.5%)
 Starting => Finishing Balance   |         

In [ ]:
#@title Training - Optimization
import psutil, os, time

%cd "/content/jesseData/{bot_name}"
Total_CPU_Cores = psutil.cpu_count()
%cd jesseData/{bot_name}
json = True #@param {type:"boolean"}
csv = True #@param {type:"boolean"}
debug = True #@param {type:"boolean"}

op_start_date = '2019-01-01' #@param {type:"date"}
op_end_date = '2020-05-31' #@param {type:"date"}
Optimal_Total = 80 #@param {type:"integer"}

cmd = "jesse optimize"

if (json):
  cmd+=" --json"

if (csv):
  cmd+=" --csv"

if (debug):
  cmd+=" --debug"

cpu = "--cpu "
cpu+= str(Total_CPU_Cores)

optimize_cmd = f"{cmd} {cpu} {op_start_date} {op_end_date} {Optimal_Total}"

print("Starting optimizing...")

!$optimize_cmd


# SAVE
###Choose any method you want to save

Recommended to sync to Drive

In [ ]:
#@title Download bot/storage to local system
#@markdown Download storage file as a zip.

#@markdown At this moment, I dont know why it download twice.
!zip -r /content/storage.zip /content/jesseData/didi27_bot/storage

from google.colab import files
files.download('/content/storage.zip')

In [ ]:
#@title Sync Colab to Drive
!rsync -aPv /content/jesseData /content/drive/MyDrive/jesse.trade/
